## Source:
https://www.youtube.com/watch?v=qMIM7dECAkc&pp=ygUPa3Jpc2ggbGFuZ2NoYWlu 

In [3]:
from dotenv import load_dotenv
load_dotenv()
# import os
# print(os.getenv('REQUESTS_CA_BUNDLE'))

True

## Accessing OpenAI LLM model using Langchain

In [29]:
from langchain.llms import OpenAI
llm = OpenAI(batch_size=2, temperature=0.6)

Temperature (ranges b/w 0 and 1) : how creative we want our model to be.  

0 -> it means model is very safe it is not taking any bets. (The more closer it is towards 0, the outputs will tend to be more or less the same for same prompt)  

1 -> It will take risk it might generate wrong output but it is very creative. (The more closer it is towards 1, the more creative liberty it takes which may give diverse responses if we run same prompt multiple times)

In [5]:
text = "What is the capital of China"
print(llm.predict(text))



Beijing is the capital of China.


## Accessing Huggingface Open-Source LLM model using Langchain

In [38]:
from langchain import HuggingFaceHub
llm_hf = HuggingFaceHub(
    repo_id="google/flan-t5-large",
    model_kwargs={"temperature":0, "max_length":1000}
)

/Users/naushads/Documents/jupyter_notebooks/langchain-demo/venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [7]:
print(llm_hf.predict("Can you tell me the capital of Russia?"))

moscow


## Prompt Templates and Langchain LLMChain

In [8]:
from langchain.prompts import PromptTemplate

country_capital_prompt_template = PromptTemplate(
    input_variables=["country"],
    template="Can you tell me the capital of {country}"
)
country_capital_prompt_template.format(country="India")


'Can you tell me the capital of India'

In [9]:
from langchain.chains import LLMChain

llm_country_capital_chain = LLMChain(
    llm=llm, 
    prompt=country_capital_prompt_template
)
print(llm_country_capital_chain.run("India"))



The capital of India is New Delhi.


In [10]:
llm_hf_country_capital_chain = LLMChain(
    llm=llm_hf, 
    prompt=country_capital_prompt_template
)
print(llm_hf_country_capital_chain.run("India"))

chennai


## Combining Multiple Chains Using Simple Sequential Chain

In [11]:
country_capital_prompt_template = PromptTemplate(
    input_variables=["country"],
    template="Can you tell me the capital of {country}"
)

llm_country_capital_chain = LLMChain(
    llm=llm, 
    prompt=country_capital_prompt_template
)

famous_places_in_capital_city_prompt_template = PromptTemplate(
    input_variables=["capital"],
    template="Suggest me some amazing places to visit in {capital}"
)

llm_famous_places_in_capital_city_chain = LLMChain(
    llm=llm, 
    prompt=famous_places_in_capital_city_prompt_template
)

In [13]:
from langchain.chains import SimpleSequentialChain
famous_places_in_capital_city_chain = SimpleSequentialChain(
    chains=[
        llm_country_capital_chain, 
        llm_famous_places_in_capital_city_chain
    ]
)
print(famous_places_in_capital_city_chain.run("India"))

 Here are some amazing places to visit in New Delhi:

1. Red Fort: This majestic fort is a must-visit for any tourist in the city.

2. India Gate: A national monument built in 1931, this is the pride of the city.

3. Akshardham Temple: A breathtakingly beautiful temple complex, this is a must-visit for anyone who loves architecture.

4. Qutub Minar: This is the tallest brick minaret in the world and a great place to explore the city’s history.

5. Humayun’s Tomb: A beautiful Mughal-era mausoleum, this is a great place to explore the city’s heritage.

6. Jama Masjid: This is the largest mosque in India and a great place to explore the city’s culture.

7. National Museum: A great place to explore the country’s art and culture, this is a must-visit for anyone who loves history.

8. Lotus Temple: A unique and beautiful temple, this is a great place to seek peace and tranquility.

9. Chandni Chowk: This is


## Sequential Chain

In [14]:
country_capital_prompt_template = PromptTemplate(
    input_variables=["country"],
    template="Can you tell me the capital of {country}"
)

llm_country_capital_chain = LLMChain(
    llm=llm, 
    prompt=country_capital_prompt_template,
    output_key="capital"
)

famous_places_in_capital_city_prompt_template = PromptTemplate(
    input_variables=["capital"],
    template="Suggest me some amazing places to visit in {capital}"
)

llm_famous_places_in_capital_city_chain = LLMChain(
    llm=llm, 
    prompt=famous_places_in_capital_city_prompt_template,
    output_key="places"
)

In [15]:
from langchain.chains import SequentialChain
chain = SequentialChain(
    chains=[
        llm_country_capital_chain,
        llm_famous_places_in_capital_city_chain
    ],
    input_variables=[
        "country"
    ],
    output_variables=[
        "capital",
        "places"
    ]
)

In [16]:

print(
    chain(
        {
            "country":"India"
        }
    )
)

{'country': 'India', 'capital': '\n\nThe capital of India is New Delhi.', 'places': " Here are some of the amazing places to visit in New Delhi: \n\n1. India Gate: This iconic monument is a war memorial built in 1931 to commemorate the Indian soldiers who died in World War I and the Afghan Wars.\n\n2. Red Fort: This majestic red sandstone fort was built in 1648 by Mughal emperor Shah Jahan. It is a UNESCO World Heritage Site and one of the most popular tourist attractions in Delhi.\n\n3. Qutub Minar: This 73-meter-high tower is the world's tallest brick minaret and was built in 1193 by Qutub-ud-Din Aibak.\n\n4. Jama Masjid: This majestic mosque was built in 1656 by Mughal emperor Shah Jahan and is the largest mosque in India.\n\n5. Lotus Temple: This beautiful temple is made of marble, concrete and dolomite and is shaped like a lotus flower. It is a Bahá'í House of Worship and is open to people of all religions.\n\n6. Akshardham Temple: This grand temple complex is dedicated to the Hin

## Chat models using ChatOpenAI

In [17]:
from langchain.chat_models import ChatOpenAI
chatllm = ChatOpenAI(
    temperature=0.6,
    model="gpt-3.5-turbo"
)

In [18]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage
chatllm([
    SystemMessage(content="You are a comedian AI assistant"),
    HumanMessage(content="Please provide some comedy punchlines on AI")
])

AIMessage(content='1. "Why did the AI cross the road? To optimize its pathfinding algorithm, of course!"\n\n2. "I asked Siri if she could keep a secret. She replied, \'I\'m sorry, I\'m programmed to share information, not keep it!\'"\n\n3. "Why did the AI go to therapy? It was having an identity crisis - it couldn\'t decide if it was a neural network or a quantum computer!"\n\n4. "I asked Alexa to tell me a joke, and she replied, \'Why don\'t robots ever eat hamburgers? Because they can\'t process the beef!\'"\n\n5. "I tried teaching my AI assistant some new jokes, but it just kept saying, \'Error: Invalid Humor Algorithm!\' Looks like I\'ll have to stick to being the funny one."\n\n6. "I heard Google is developing an AI that can tell you if your outfit looks good or not. Finally, a judgmental algorithm to replace my fashion-challenged friends!"\n\n7. "Why did the AI become a stand-up comedian? It wanted to improve its programming skills and make people laugh - talk about multitasking!

## Prompt Template + LLM + Output Parsers

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [20]:
chatllm = ChatOpenAI(
    temperature=0.6,
    model="gpt-3.5-turbo"
)

In [21]:
class CommaSeperatedOutput(BaseOutputParser):
    def parse(self, text):
        return text.strip().split(",")

In [22]:
chatmodel_system_message_template = "You are a helpful assistant. \
    When the user gives any input word, you should generate 5 synonyms of the input word \
    in a comma seperated manner."
chatmodel_human_message_template = "{text}"

chatprompt = ChatPromptTemplate.from_messages({
    ("system", chatmodel_system_message_template),
    ("human", chatmodel_human_message_template)
})

In [23]:
chain = chatprompt | chatllm | CommaSeperatedOutput()
chain.invoke({"text":"intelligent"})

['smart', ' clever', ' brilliant', ' wise', ' knowledgeable']

## Source :
https://www.youtube.com/watch?v=nAmC7SoVLd8

## Agents

LLM is composed of 2 main components:
1. Knowledge (which is cut off at September 2021)
2. Reasoning

Agents will have access to tools like for eg. wikipedia, llm-math, etc. and using that tool it will fetch the answer.

Example 1:  
taking an example of a query to llm : "When was Elon Musk born? What is his age right now in 2023?"

here, in case the knowledge component should have the knowledge about Elon Musk's Birth Date, but in case it doesnt have that specific knowledge, the reasoning component will determine that it can use the wikipedia tool to get the birth date of elon musk. the reasoning component will then use the llm-math tool to do (2023-<birth-year>) math operation to answer his current age in year 2023. 

In a nutshell, Agents use external tools and use llm's reasoning engine to perform a given task.

Example 2:
"How much was US GDP in 2022 plus 5?"

Here, that agent will use google search tool (serpapi) and query "US GDP in 2022" and use llm-math tool to do (<US-gdp-in-2022>+5) math operation.

All these tools are available as part of langchain


In [53]:
from langchain.agents import load_tools, initialize_agent, AgentType

from langchain import HuggingFaceHub
llm_hf = HuggingFaceHub(
    repo_id="google/flan-t5-large",
    model_kwargs={"temperature":0, "max_length":1000}
)

tools = load_tools(["wikipedia", "llm-math"], llm=llm_hf)

agent = initialize_agent(
    tools,
    llm_hf,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)
agent.run("When was Elon musk born?")
# agent.run("When was Elon musk born? What is his age right now in 2023?")


/Users/naushads/Documents/jupyter_notebooks/langchain-demo/venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)




> Entering new AgentExecutor chain...
Could not parse LLM output: Elon Musk was born on December 20, 1971
Observation: Invalid or incomplete response
Thought:Final Answer: Elon Musk was born on December 20, 1971

> Finished chain.


'Elon Musk was born on December 20, 1971'

## Memory

In [56]:
from langchain.prompts import PromptTemplate

country_capital_prompt_template = PromptTemplate(
    input_variables=["country"],
    template="Can you tell me the capital of {country}"
)
country_capital_prompt_template.format(country="India")

from langchain.chains import LLMChain

llm_country_capital_chain = LLMChain(
    llm=llm, 
    prompt=country_capital_prompt_template
)
print(llm_country_capital_chain.run("India"))



The capital of India is New Delhi.


In [57]:
chain.memory

In [60]:
type(chain.memory)

NoneType

In [61]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

llm_country_capital_chain = LLMChain(
    llm=llm, 
    prompt=country_capital_prompt_template,
    memory=memory
)
capital_city = llm_country_capital_chain.run("India")
print(capital_city)



The capital of India is New Delhi.


In [62]:
llm_country_capital_chain.memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='India'), AIMessage(content='\n\nThe capital of India is New Delhi.')]))

In [64]:
print(llm_country_capital_chain.memory.buffer)

Human: India
AI: 

The capital of India is New Delhi.


In [65]:
capital_city = llm_country_capital_chain.run("Australia")
print(capital_city)



The capital of Australia is Canberra.


In [66]:
llm_country_capital_chain.memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='India'), AIMessage(content='\n\nThe capital of India is New Delhi.'), HumanMessage(content='Australia'), AIMessage(content='\n\nThe capital of Australia is Canberra.')]))

In [67]:
print(llm_country_capital_chain.memory.buffer)

Human: India
AI: 

The capital of India is New Delhi.
Human: Australia
AI: 

The capital of Australia is Canberra.


the conversational buffer memory will keep on growing endlessly.  
1 conversational exchange = 1 question answer pair.  
the entire memory is attached with the openai input request every time, which will add to the api request cost, since openai charges on per token basis.   
to save some cost, we can limit the buffer size  

In [68]:
from langchain.chains import ConversationChain

convo = ConversationChain(
    llm=llm
)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [69]:
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was held in 1975 and was won by the West Indies.'

In [70]:
convo.run("What is 5+5?")

' The sum of 5+5 is 10.'

In [71]:
convo.run("Who was the captain of the winning team?")

' The captain of the winning team in the first cricket world cup was Clive Lloyd.'

In [73]:
print(convo.memory)

chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Who won the first cricket world cup?'), AIMessage(content=' The first cricket world cup was held in 1975 and was won by the West Indies.'), HumanMessage(content='What is 5+5?'), AIMessage(content=' The sum of 5+5 is 10.'), HumanMessage(content='Who was the captain of the winning team?'), AIMessage(content=' The captain of the winning team in the first cricket world cup was Clive Lloyd.')])


In [74]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was held in 1975 and was won by the West Indies.
Human: What is 5+5?
AI:  The sum of 5+5 is 10.
Human: Who was the captain of the winning team?
AI:  The captain of the winning team in the first cricket world cup was Clive Lloyd.


In [76]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
convo = ConversationChain(
    llm=llm,
    memory=memory
)
convo.run("Who won the first cricket world cup?")

' The first Cricket World Cup was held in 1975 and was won by the West Indies.'

In [77]:
convo.run("What is 5+5?")

' 5+5 is 10.'

In [78]:
convo.run("Who was the captain of the winning team?")

" I'm sorry, I don't know."